# How many names are found in a given version of the Bible in the places where the macula data says they are?

In [1]:
#!/usr/bin/env python3

import csv
from collections import Counter
import json
import numpy as np
import os
import pandas as pd
import re
from pathlib import Path
import sys
import urllib.request

In [2]:
data_folder = Path("D:/GitHub/davidbaines/trabina/data")

by_lang_folder = data_folder / "by-lang"
compare_col = "English gloss"
matched_names_tsv = data_folder / "matched_eng_web_macula_names.tsv"
macula_json_file = Path("D:/GitHub/davidbaines/trabina") / "macula.json"

bible_url = r"https://raw.githubusercontent.com/BibleNLP/ebible-corpus/main/corpus/eng-eng-web.txt"

In [9]:
def get_vrefs(silnlp_vref_file):    
    # Get the silnlp references to line numbers:
    with open(silnlp_vref_file, 'r', encoding='utf-8') as vrefs_file:
        vrefs_dict = {ref.strip('\n'): i+1 for i, ref in enumerate(vrefs_file.readlines())}
    
    #print(f"vrefs_dict is {vrefs_dict}")
    #try :
    #    print(f"vrefs dict contains Mark 16:99? {vrefs_dict['MRK 16:99']}")
    #except KeyError:
    #    print(f"vrefs dict doesn't contain Mark 16:99.")
    vrefs = pd.DataFrame([vrefs_dict]).T
    vrefs.rename({0:'silnlp_line_number'}, axis='columns', inplace=True)

    # To convert reference to line number get the 1st (index 0) element of the vrefs for that reference. E.g.:
    #print(vrefs.loc['ENO 1:2']['silnlp_line_number'])
    return vrefs

silnlp_assets_folder = Path("D:/GitHub/davidbaines/trabina/silnlp/assets")
silnlp_vref_file = silnlp_assets_folder / "vref.txt"
vrefs = get_vrefs(silnlp_vref_file)

In [3]:
macula = pd.read_json(macula_json_file)
macula

,refs,Source,English gloss,Hebrew source,English gloss of Hebrew,Greek gloss of Hebrew,Mandarin gloss of Hebrew,Greek source,Greek lemma,Greek normalized,English gloss of Greek,book,chapter_no,verse_no,word_no
0,GEN 2:4!8,יְהוָ֥ה,LORD,יְהוָ֥ה,LORD,,耶和华,,,,,GEN,2.0,4.0,8.0
1,GEN 2:5!15,יְהוָ֤ה,LORD,יְהוָ֤ה,LORD,,耶和华,,,,,GEN,2.0,5.0,15.0
2,GEN 2:7!2,יְהוָ֨ה,LORD,יְהוָ֨ה,LORD,,耶和华,,,,,GEN,2.0,7.0,2.0
3,GEN 2:8!2,יְהוָ֧ה,LORD,יְהוָ֧ה,LORD,κύριος,耶和华,,,,,GEN,2.0,8.0,2.0
4,GEN 2:8!5,עֵ֖דֶן,Eden,עֵ֖דֶן,Eden,εδεμ,伊甸,,,,,GEN,2.0,8.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38814,REV 22:13!6,"Ὦ,",Omega,,,,,"Ὦ,",Ὦ,Ὦ,Omega,REV,22.0,13.0,6.0
38815,REV 22:16!2,Ἰησοῦς,Jesus,,,,,Ἰησοῦς,Ἰησοῦς,Ἰησοῦς,Jesus,REV,22.0,16.0,2.0
38816,REV 22:16!20,"Δαυείδ,",of David,,,,,"Δαυείδ,",Δαυίδ,Δαυείδ,of David,REV,22.0,16.0,20.0
38817,REV 22:20!11,Ἰησοῦ.,Jesus,,,,,Ἰησοῦ.,Ἰησοῦς,Ἰησοῦ,Jesus,REV,22.0,20.0,11.0


In [4]:
print(38819 - 38621)
print(macula[macula['English gloss'] != ''])


198
               refs   Source English gloss Hebrew source  \
0         GEN 2:4!8  יְהוָ֥ה          LORD       יְהוָ֥ה   
1        GEN 2:5!15  יְהוָ֤ה          LORD       יְהוָ֤ה   
2         GEN 2:7!2  יְהוָ֨ה          LORD       יְהוָ֨ה   
3         GEN 2:8!2  יְהוָ֧ה          LORD       יְהוָ֧ה   
4         GEN 2:8!5   עֵ֖דֶן          Eden        עֵ֖דֶן   
...             ...      ...           ...           ...   
38814   REV 22:13!6       Ὦ,         Omega                 
38815   REV 22:16!2   Ἰησοῦς         Jesus                 
38816  REV 22:16!20  Δαυείδ,      of David                 
38817  REV 22:20!11   Ἰησοῦ.         Jesus                 
38818   REV 22:21!5    Ἰησοῦ         Jesus                 

      English gloss of Hebrew Greek gloss of Hebrew Mandarin gloss of Hebrew  \
0                        LORD                                            耶和华   
1                        LORD                                            耶和华   
2                        LORD      

In [5]:
def get_bible_text(filename: str) -> list:
    """
    Reads a text file from a local file or URL, returns a list of lines.
    Inputs:
        filename:  URL or local filepath
    Outputs:
        :      Dictionary mapping semantic domains to descriptions
    """
    if filename[:4] == 'http':
        f = urllib.request.urlopen(filename)
        lines =  [line.decode('utf-8').strip('\n') for line in f.readlines()]
    else:
        with open(filename, 'r', encoding='utf-8') as f:
            lines = [line.strip('\n') for line in f.readlines()]
    return lines

In [6]:
# Read in a Bible
bible_lines = get_bible_text(bible_url)    
print(bible_lines[:10])
bible_dict = {vref + 1 : verse for vref,verse in enumerate(bible_lines)}
#print(bible_dict)

# Make a Dataframe from the dictionary
bible = pd.DataFrame.from_dict(bible_dict, orient='index', dtype=str, columns=['verse'])
print(bible.verse == '')
#print()
#print(hebrew_ot[hebrew_ot.verse == ''])
#print()
#print(hebrew_ot[hebrew_ot.verse == ''].index)

#hebrew_ot.drop(hebrew_ot['verse']=='', inplace=True)
bible.drop(bible[bible.verse == ''].index, inplace=True)
bible


['In the beginning, God created the heavens and the earth.', 'The earth was formless and empty. Darkness was on the surface of the deep and God’s Spirit was hovering over the surface of the waters.', 'God said, “Let there be light,” and there was light.', 'God saw the light, and saw that it was good. God divided the light from the darkness.', 'God called the light “day”, and the darkness he called “night”. There was evening and there was morning, the first day.', 'God said, “Let there be an expanse in the middle of the waters, and let it divide the waters from the waters.”', 'God made the expanse, and divided the waters which were under the expanse from the waters which were above the expanse; and it was so.', 'God called the expanse “sky”. There was evening and there was morning, a second day.', 'God said, “Let the waters under the sky be gathered together to one place, and let the dry land appear;” and it was so.', 'God called the dry land “earth”, and the gathering together of the w

,verse
1,"In the beginning, God created the heavens and ..."
2,The earth was formless and empty. Darkness was...
3,"God said, “Let there be light,” and there was ..."
4,"God saw the light, and saw that it was good. G..."
5,"God called the light “day”, and the darkness h..."
...,...
37832,"Who shall tell my Lord? The Lord himself, he h..."
37833,He sent forth his angel and took me from my fa...
37834,My brothers were handsome and tall; but the Lo...
37835,"I went out to meet the Philistine, and he curs..."


In [14]:
# Add silnlp_line_numbers to macula data
print(macula)
macula = pd.merge(macula, vrefs, how='left', left_on='ref_only', right_index=True)
macula.silnlp_line_number = macula.silnlp_line_number.fillna(0).astype(int)

               refs   Source English gloss Hebrew source  \
0         GEN 2:4!8  יְהוָ֥ה          LORD       יְהוָ֥ה   
1        GEN 2:5!15  יְהוָ֤ה          LORD       יְהוָ֤ה   
2         GEN 2:7!2  יְהוָ֨ה          LORD       יְהוָ֨ה   
3         GEN 2:8!2  יְהוָ֧ה          LORD       יְהוָ֧ה   
4         GEN 2:8!5   עֵ֖דֶן          Eden        עֵ֖דֶן   
...             ...      ...           ...           ...   
38814   REV 22:13!6       Ὦ,         Omega                 
38815   REV 22:16!2   Ἰησοῦς         Jesus                 
38816  REV 22:16!20  Δαυείδ,      of David                 
38817  REV 22:20!11   Ἰησοῦ.         Jesus                 
38818   REV 22:21!5    Ἰησοῦ         Jesus                 

      English gloss of Hebrew Greek gloss of Hebrew Mandarin gloss of Hebrew  \
0                        LORD                                            耶和华   
1                        LORD                                            耶和华   
2                        LORD          

KeyError: 'ref_only'

In [11]:
# For each English gloss, does it appear in the verse?
# Add the verse to the macula data
macula = pd.merge(macula, bible, how='left', left_on = 'silnlp_line_number', right_index=True)
macula

KeyError: 'silnlp_line_number'

In [ ]:
macula.fillna('').apply(lambda row: row['English gloss'].lower() in row.verse.lower(), axis=1)

In [ ]:
# print([x[0].lower() in str(x[1]).lower() for x in zip(macula['English gloss'], macula['verse'])][:100])
# macula['English_gloss_in_eng_web'] = [x[0].lower() in str(x[1]).lower() for x in zip(macula['English gloss'], macula['verse'])]
macula['English_gloss_in_eng_web'] = macula.fillna('').apply(lambda row: row['English gloss'].lower() in row.verse.lower(), axis=1)
macula

In [ ]:
# From https://www.statology.org/pandas-groupby-count-with-condition/
#groupby team and count number of 'pos' equal to 'Gu'
#df_count = df.groupby('team')['pos'].apply(lambda x: (x=='Gu').sum()).reset_index(name='count')
bible_wordcount_found = macula.groupby('English gloss')['English_gloss_in_eng_web'].apply(lambda x: (x).sum()).reset_index(name='Found')
bible_wordcount_not_found = macula.groupby('English gloss')['English_gloss_in_eng_web'].apply(lambda x: (x == False).sum()).reset_index(name='Not_found')
print(bible_wordcount_found)
bible_wordcount = pd.merge(bible_wordcount_found,bible_wordcount_not_found)
print(bible_wordcount)
bible_wordcount['Found_ratio'] = bible_wordcount.apply(lambda x: int(x[1]) / (int(x[1]) + int(x[2])), axis=1)  


In [ ]:
bible_wordcount.to_csv(matched_names_tsv, sep = '\t')